### Import Library

In [41]:
# Import library yang diperlukan
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam


#ignore warning
import warnings
warnings.filterwarnings('ignore')

### Direktori

In [42]:
# Membuat path join untuk direktori dataset
dataset_dir = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/'

# Menambahkan path join untuk direktori pelatihan
train_dir = os.path.join(dataset_dir, 'pengujian/')

# Menambahkan path join untuk direktori pengujian
test_dir = os.path.join(dataset_dir, 'pelatihan/')

### Split Data

In [43]:
import os

# Membuat path join untuk direktori dataset
dataset_dir = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/'

# Menambahkan path join untuk direktori pelatihan (70%)
train_dir = os.path.join(dataset_dir, 'pengujian/')

# Menambahkan path join untuk direktori pengujian (30%)
test_dir = os.path.join(dataset_dir, 'pelatihan/')

# Menentukan porsi data citra untuk pelatihan dan pengujian
train_ratio = 0.7
test_ratio = 0.3

# Mendapatkan list file citra dalam direktori dataset
citra_files = [file for file in os.listdir(dataset_dir) if file.endswith('.JPG') or file.endwith('.png')]

# Menghitung jumlah data untuk pelatihan dan pengujian
num_train = int(len(citra_files) * train_ratio)
num_test = int(len(citra_files) * test_ratio)

# Memisahkan file citra untuk pelatihan
train_citra_files = citra_files[:num_train]
train_citra_dir = os.path.join(train_dir, 'citra/')

# Memisahkan file citra untuk pengujian
test_citra_files = citra_files[num_train:num_train + num_test]
test_citra_dir = os.path.join(test_dir, 'citra/')

# Membuat path join untuk direktori citra pelatihan dan pengujian
train_citra_paths = [os.path.join(train_citra_dir, file) for file in train_citra_files]
test_citra_paths = [os.path.join(test_citra_dir, file) for file in test_citra_files]

# Output paths
print("Path citra untuk pelatihan (70%):")
for path in train_citra_paths:
    print(path)

print("\nPath citra untuk pengujian (30%):")
for path in test_citra_paths:
    print(path)


Path citra untuk pelatihan (70%):
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_A3772FV.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_A4638EG.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_AA2503XT.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_AA2677GV.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_AA2930VV.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_AA2956JY.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_AA314FD.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/pengujian/citra/K1_AA3189LY.JPG
D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra